## **Analyze the sentiment of reviews**

In [1]:
WITH cte AS (
    SELECT id, comments, azure_cognitive.analyze_sentiment(comments, 'en') AS sentiment FROM reviews
)
SELECT
    id,
    (sentiment).sentiment,
    (sentiment).positive_score,
    (sentiment).neutral_score,
    (sentiment).negative_score,
    comments
FROM cte
WHERE (sentiment).positive_score > 0.98
LIMIT 5;

(5 row(s) affected)

Total execution time: 00:00:05.171

id,sentiment,positive_score,neutral_score,negative_score,comments
3,positive,1,0,0,One of the most pleasant stays I've had in my travels. The host is wonderful!
4,positive,1,0,0,"The host's suite is beautiful, cozy and conveniently located. I love that armchair! One tip: when she offers you detailed directions: take them, especially if you're arriving after dark! ;)"
6,positive,0.99,0.01,0,I had a wonderful 4 night stay in this apartment. The place is crisp and clean and in a fantastic location. Walking around the Queen Anne neighborhood each day showed the real charm of Seattle and I hope I have a chance to stay again!
16,positive,1,0,0,I was very comfortable at the apartment. A great overall experience and she really went out of her way to provide some good recommendations.
18,positive,1,0,0,We absolutely adored this lovely apartment! It was the perfect spot to help us ease into the Seattle community upon our relocation to Seattle from N.Y. while waiting for our things to arrive...


## Store Sentiment in the Reviews Table ( Performance!! )

In [5]:
ALTER TABLE reviews
ADD COLUMN sentiment varchar(10),
ADD COLUMN positive_score numeric,
ADD COLUMN neutral_score numeric,
ADD COLUMN negative_score numeric;

WITH cte AS (
    SELECT id, azure_cognitive.analyze_sentiment(comments, 'en') AS sentiment FROM reviews
)
UPDATE reviews AS r
SET
    sentiment = (cte.sentiment).sentiment,
    positive_score = (cte.sentiment).positive_score,
    neutral_score = (cte.sentiment).neutral_score,
    negative_score = (cte.sentiment).negative_score
FROM cte
WHERE r.id = cte.id;

UPDATE 354

Total execution time: 00:03:06.597

## <span style="font-size: 14px;"><b>The same in a batch of 10.</b></span>

In [6]:
WITH cte AS (
    SELECT azure_cognitive.analyze_sentiment(ARRAY(SELECT comments FROM reviews ORDER BY id), 'en', batch_size => 10) as sentiments
),
sentiment_cte AS (
    SELECT
        ROW_NUMBER() OVER () AS id,
        sentiments AS sentiment
    FROM cte
)
UPDATE reviews AS r
SET
    sentiment = (sentiment_cte.sentiment).sentiment,
    positive_score = (sentiment_cte.sentiment).positive_score,
    neutral_score = (sentiment_cte.sentiment).neutral_score,
    negative_score = (sentiment_cte.sentiment).negative_score
FROM sentiment_cte
WHERE r.id = sentiment_cte.id;

UPDATE 354

Total execution time: 00:00:06.648

In [7]:
SELECT
    id,
    negative_score,
    comments
FROM reviews
WHERE sentiment = 'negative'
ORDER BY negative_score DESC;

(8 row(s) affected)

Total execution time: 00:00:00.206

id,negative_score,comments
332,0.98,The host canceled this reservation 3 days before arrival without first talking to us. We were left stranded in Seattle with no place to stay. Would not recommend.
303,0.97,The host canceled this reservation 1 day before arrival. We were left stranded in Seattle with no place to stay. Would not recommend.
129,0.97,The reservation was canceled by the host 1 days before arrival. This left us scrambling for a place to stay at the last minute. Not cool! Won't book with this host again.
301,0.95,The host canceled this reservation 6 days before arrival. We were left stranded in Seattle with no place to stay. Would not recommend.
59,0.92,Lovely place but very noisy in the evening. Not to recommend to people who do not want their sleep to be affected.
252,0.83,"Arrived to find an apartment on the lower west side of Queen Anne with views of Interbay (15th ave. west), the train tracks, and Magnolia hill.. The building is undergoing what I would call a major, not a minor, renovation at this time. Apartment is west-facing, so we found the space stuffy to be quite honest. Also, the construction, being loud with hammering and nail guns starting at 7:30 am forced us to leave for one full day of our stay between the hours of 7:30-4:30."
317,0.82,"Great neighborhood, but the \patio\ photo was misleading. There was no table and chair(s) - you have to climb out a window onto a roof that can be accessed by several other windows, so limited privacy.It would have been helpful to know before booking that there was no coffee maker, toaster, or microwave and that parking can be difficult.Comfy bed, though."
268,0.53,"The good: Good location, comfortable space, comfortable bed, washer and dryerThe bad: It smelled musty (it's an exposed basement), street noise is loud, there were many cobwebs, the only bar soap was soap used by previous tenants (I did bring our own just in case), very few towels, found bathroom floor mats in laundry room which seemed clean, found coffee maker in laundry room (after we bought one)."
